# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [1]:
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload

from concept_dependency_graph import ConceptDependencyGraph
import data_generator as dg
import simple_mdp as sm

In [18]:
n_concepts = 5
concept_tree = sm.create_custom_dependency()
n_students = 10000
seqlen = 3
policy = 'random'
filename = 'test-n{}-l{}-{}.pickle'.format(n_students, seqlen, policy)

In [19]:
print ("Initializing synthetic data sets...")
dg.generate_data(concept_tree, n_students=n_students, seqlen=seqlen, policy=policy, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 10000 students with behavior policy random and sequence length 3.
Data generation completed. 


In [20]:
import dynamics_model_class as dmc
import numpy as np
import dataset_utils

In [21]:
# test_model hidden=16
# test_model_small hidden=5
dmodel = dmc.DynamicsModel(model_id="test_model_small", timesteps=seqlen, load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test_model_small/
Checkpoint directory path: ../checkpoints/test_model_small/
Model loaded.


In [23]:
# load toy data
data = dataset_utils.load_data(filename='{}{}'.format(dg.SYN_DATA_DIR, filename))
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

In [24]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])

In [26]:
dmodel.train(train_data, n_epoch=5, load_checkpoint=True)

Training Step: 2114  | total loss: 0.01659 | time: 1.594s
| Adam | epoch: 015 | loss: 0.01659 -- iter: 8960/9000
Training Step: 2115  | total loss: 0.01656 | time: 2.611s
| Adam | epoch: 015 | loss: 0.01656 | val_loss: 0.01747 -- iter: 9000/9000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [27]:
preds = dmodel.predict(input_data_[:1,:2, :])
print len(preds)
print len(preds[0])
print preds[0]

1
3
[[0.714431881904602, 0.5344692468643188, 0.5211542248725891, 0.07327841967344284, 0.05542478710412979], [0.8769022822380066, 0.6045227646827698, 0.5635643005371094, 0.04714758321642876, 0.05296757444739342], [0.0, 0.0, 0.0, 0.0, 0.0]]


In [28]:
rnnsim = dmc.RnnStudentSim(dmodel)

In [29]:
from mcts import *
actix = 0
actvec = np.zeros((n_concepts,))
actvec[actix] = 1
act = StudentAction(actix, actvec)
print rnnsim.sequence
print rnnsim.sample_observations()

[]
None


In [32]:
rnnsim.advance_simulator(act, 1)


In [33]:
print rnnsim.sequence
print rnnsim.sample_observations()
print rnnsim.sample_reward()

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]
[0.8769068121910095, 0.6046217679977417, 0.563060462474823, 0.04691258817911148, 0.05182706192135811]
2.14332869276
